In [1]:
import pandas as pd
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from numpy import genfromtxt
from time import time
from datetime import datetime
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
import requests

## Data scraping
Данные взяты с сайта https://fbref.com/en/comps/9/Premier-League-Stats

In [2]:
spec_name=[]
spec_item1=[]
spec_item2=[]
spec_item3=[]
spec_item4=[]
spec_item5=[]
spec_item6=[]
spec_item7=[]
spec_item8=[]
spec_item9=[]
spec_item10=[]
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
base_url = "https://fbref.com/en/comps/9/Premier-League-Stats"
driver.get(base_url)
tables = WebDriverWait(driver, 5).until(EC.presence_of_all_elements_located((By.XPATH,"//div[@id='div_results111601_overall']//table[@id='results111601_overall']")))
for table in tables:
    for row in table.find_elements(by=By.XPATH, value=".//tbody//tr"):
        spec_name.append(row.find_element(by=By.XPATH, value=".//td//a").get_attribute('textContent'))
        spec_item1.append(row.find_element(by=By.XPATH, value='.//td[2]').get_attribute('textContent'))
        spec_item2.append(row.find_element(by=By.XPATH, value='.//td[3]').get_attribute('textContent'))
        spec_item3.append(row.find_element(by=By.XPATH, value='.//td[4]').get_attribute('textContent'))
        spec_item4.append(row.find_element(by=By.XPATH, value='.//td[5]').get_attribute('textContent'))
        spec_item5.append(row.find_element(by=By.XPATH, value='.//td[6]').get_attribute('textContent'))
        spec_item6.append(row.find_element(by=By.XPATH, value='.//td[7]').get_attribute('textContent'))
        spec_item7.append(row.find_element(by=By.XPATH, value='.//td[8]').get_attribute('textContent'))
        spec_item8.append(row.find_element(by=By.XPATH, value='.//td[9]').get_attribute('textContent'))
        spec_item9.append(row.find_element(by=By.XPATH, value='.//td[11]').get_attribute('textContent'))
        spec_item10.append(row.find_element(by=By.XPATH, value='.//td[15]').get_attribute('textContent'))
df = pd.DataFrame({
    "Club Name":spec_name, "Matches Played":spec_item1, "Wins":spec_item2, 'Draws':spec_item3, 
     'Loses':spec_item4, 'Goals For':spec_item5, 'Goals Against': spec_item6, 'Goal Difference': spec_item7, 
     'Points':spec_item8, 'Expected Goals':spec_item9, 'Attendance':spec_item10
})
                   

print(df)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 102.0.5005
[WDM] - Get LATEST chromedriver version for 102.0.5005 google-chrome
[WDM] - Driver [C:\Users\timur\.wdm\drivers\chromedriver\win32\102.0.5005.61\chromedriver.exe] found in cache


          Club Name Matches Played Wins Draws Loses Goals For Goals Against  \
0   Manchester City             38   29     6     3        99            26   
1         Liverpool             38   28     8     2        94            26   
2           Chelsea             38   21    11     6        76            33   
3         Tottenham             38   22     5    11        69            40   
4           Arsenal             38   22     3    13        61            48   
5    Manchester Utd             38   16    10    12        57            57   
6          West Ham             38   16     8    14        60            51   
7    Leicester City             38   14    10    14        62            59   
8          Brighton             38   12    15    11        42            44   
9            Wolves             38   15     6    17        38            43   
10    Newcastle Utd             38   13    10    15        44            62   
11   Crystal Palace             38   11    15    12 

## Исправим некоторые названия клубов для однородности данных, а также приведем значения столбца с посещаемостью в числовой вид с помощью регулярного выражения. 

In [3]:
df.at[5, 'Club Name'] = 'Manchester United'
df.at[3, 'Club Name'] = 'Tottenham Hotspur'
df.at[6, 'Club Name'] = 'West Ham United'
df.at[8, 'Club Name'] = 'Brighton & Hove Albion'
df.at[10, 'Club Name'] = 'Newcastle United'
df.at[9, 'Club Name'] = 'Wolverhampton Wanderers'
df['Attendance'] = pd.to_numeric(df['Attendance'].str.replace(',', '')) 
df

,Club Name,Matches Played,Wins,Draws,Loses,Goals For,Goals Against,Goal Difference,Points,Expected Goals,Attendance
0,Manchester City,38,29,6,3,99,26,+73,93,89.0,52774
1,Liverpool,38,28,8,2,94,26,+68,92,89.2,53352
2,Chelsea,38,21,11,6,76,33,+43,74,67.2,36443
3,Tottenham Hotspur,38,22,5,11,69,40,+29,71,65.1,56523
4,Arsenal,38,22,3,13,61,48,+13,69,59.9,59665
5,Manchester United,38,16,10,12,57,57,0,58,54.8,73150
6,West Ham United,38,16,8,14,60,51,+9,56,49.9,58370
7,Leicester City,38,14,10,14,62,59,+3,52,50.8,32493
8,Brighton & Hove Albion,38,12,15,11,42,44,-2,51,46.0,30966
9,Wolverhampton Wanderers,38,15,6,17,38,43,-5,51,35.9,30841


## Возьмем данные с расположением стадионов для некоторых клубов мира и немного их подправим

In [4]:
df2=pd.read_csv('./cities.csv')
df2 = df2.applymap(lambda x: str(x).strip())
df2

,Team,FDCOUK,City,Stadium,Capacity,Latitude,Longitude,Country
0,Arsenal,Arsenal,London,Emirates Stadium,60361,51.555,-0.108611,England
1,Aston Villa,Aston Villa,Birmingham,Villa Park,42785,52.509167,-1.884722,England
2,Blackburn Rovers,Blackburn,Blackburn,Ewood Park,31154,53.728611,-2.489167,England
3,Bolton Wanderers,Bolton,Bolton,Reebok Stadium,28100,53.580556,-2.535556,England
4,Chelsea,Chelsea,London,Stamford Bridge,42449,51.481667,-0.191111,England
...,...,...,...,...,...,...,...,...
131,Auxerre,Auxerre,Auxerre,Stade de l'Abb?-Deschamps,24493,47.786753,3.588664,France
132,Brest,Brest,Brest,Stade Francis-Le Bl?,16000,48.402932,-4.461694,France
133,Caen,Caen,Caen,Stade Michel d'Ornano,21500,49.179461,-0.396767,France
134,Sporting de Gij?n,Sp Gijon,Gij?n,El Molin?n,30000,43.536111,-5.637222,Spain


## Создададим новый датафрейм, в котором будут находиться лишь интересующие нас клубы

In [5]:
filtered_df2 = df2[df2['Team'].isin(df['Club Name'])]
filtered_df2

,Team,FDCOUK,City,Stadium,Capacity,Latitude,Longitude,Country
0,Arsenal,Arsenal,London,Emirates Stadium,60361,51.555,-0.108611,England
1,Aston Villa,Aston Villa,Birmingham,Villa Park,42785,52.509167,-1.884722,England
4,Chelsea,Chelsea,London,Stamford Bridge,42449,51.481667,-0.191111,England
5,Everton,Everton,Liverpool,Goodison Park,40157,53.438889,-2.966389,England
7,Liverpool,Liverpool,Liverpool,Anfield,45276,53.430819,-2.960828,England
8,Manchester City,Man City,Manchester,Etihad Stadium,47405,53.482989,-2.200292,England
9,Manchester United,Man United,Stretford,Old Trafford,75811,53.463056,-2.291389,England
10,Newcastle United,Newcastle,Newcastle upon Tyne,Sports Direct Arena,52409,54.975556,-1.621667,England
11,Norwich City,Norwich,Norwich,Carrow Road,27010,52.622128,1.308653,England
16,Tottenham Hotspur,Tottenham,London,White Hart Lane,36230,51.603333,-0.065833,England


## Добавим недостающий клуб вручную и отсортируем наши датафреймы по алфавиту

In [6]:
new_row = {'Team': 'Brentford', 'FDCOUK': 'Brentford', 'City': 'London', 'Stadium': 'Brentford Community Stadium', 'Capacity': 17250, 'Latitude': 51.489809, 'Longitude': -0.287871, 'Country': 'England'}
filtered_df2 = filtered_df2.append(new_row, ignore_index=True)
filtered_df2 = filtered_df2.sort_values('Team')
filtered_df2

,Team,FDCOUK,City,Stadium,Capacity,Latitude,Longitude,Country
0,Arsenal,Arsenal,London,Emirates Stadium,60361,51.555,-0.108611,England
1,Aston Villa,Aston Villa,Birmingham,Villa Park,42785,52.509167,-1.884722,England
19,Brentford,Brentford,London,Brentford Community Stadium,17250,51.489809,-0.287871,England
11,Brighton & Hove Albion,Brighton,Brighton,American Express Community Stadium,22374,50.861822,-0.083278,England
12,Burnley,Burnley,Burnley,Turf Moor,22546,53.789167,-2.230278,England
2,Chelsea,Chelsea,London,Stamford Bridge,42449,51.481667,-0.191111,England
13,Crystal Palace,Crystal Palace,London,Selhurst Park,26309,51.398333,-0.085556,England
3,Everton,Everton,Liverpool,Goodison Park,40157,53.438889,-2.966389,England
14,Leeds United,Leeds,Leeds,Elland Road,39460,53.777778,-1.572222,England
15,Leicester City,Leicester,Leicester,King Power Stadium,32500,52.620278,-1.142222,England


In [7]:
filtered_df = df.sort_values('Club Name')
filtered_df

,Club Name,Matches Played,Wins,Draws,Loses,Goals For,Goals Against,Goal Difference,Points,Expected Goals,Attendance
4,Arsenal,38,22,3,13,61,48,+13,69,59.9,59665
13,Aston Villa,38,13,6,19,52,54,-2,45,44.0,41681
12,Brentford,38,13,7,18,48,56,-8,46,47.8,16918
8,Brighton & Hove Albion,38,12,15,11,42,44,-2,51,46.0,30966
17,Burnley,38,7,14,17,34,53,-19,35,39.0,19278
2,Chelsea,38,21,11,6,76,33,+43,74,67.2,36443
11,Crystal Palace,38,11,15,12,50,46,+4,48,46.4,24588
15,Everton,38,11,6,21,43,66,-23,39,42.8,38446
16,Leeds United,38,9,11,18,42,79,-37,38,47.2,36308
7,Leicester City,38,14,10,14,62,59,+3,52,50.8,32493


## Создадим объединенный датафрейм и удалим из него избыточную информацию

In [8]:
merged_dataframe = filtered_df.merge(filtered_df2, left_on='Club Name', right_on='Team')
merged_dataframe

,Club Name,Matches Played,Wins,Draws,Loses,Goals For,Goals Against,Goal Difference,Points,Expected Goals,Attendance,Team,FDCOUK,City,Stadium,Capacity,Latitude,Longitude,Country
0,Arsenal,38,22,3,13,61,48,+13,69,59.9,59665,Arsenal,Arsenal,London,Emirates Stadium,60361,51.555,-0.108611,England
1,Aston Villa,38,13,6,19,52,54,-2,45,44.0,41681,Aston Villa,Aston Villa,Birmingham,Villa Park,42785,52.509167,-1.884722,England
2,Brentford,38,13,7,18,48,56,-8,46,47.8,16918,Brentford,Brentford,London,Brentford Community Stadium,17250,51.489809,-0.287871,England
3,Brighton & Hove Albion,38,12,15,11,42,44,-2,51,46.0,30966,Brighton & Hove Albion,Brighton,Brighton,American Express Community Stadium,22374,50.861822,-0.083278,England
4,Burnley,38,7,14,17,34,53,-19,35,39.0,19278,Burnley,Burnley,Burnley,Turf Moor,22546,53.789167,-2.230278,England
5,Chelsea,38,21,11,6,76,33,+43,74,67.2,36443,Chelsea,Chelsea,London,Stamford Bridge,42449,51.481667,-0.191111,England
6,Crystal Palace,38,11,15,12,50,46,+4,48,46.4,24588,Crystal Palace,Crystal Palace,London,Selhurst Park,26309,51.398333,-0.085556,England
7,Everton,38,11,6,21,43,66,-23,39,42.8,38446,Everton,Everton,Liverpool,Goodison Park,40157,53.438889,-2.966389,England
8,Leeds United,38,9,11,18,42,79,-37,38,47.2,36308,Leeds United,Leeds,Leeds,Elland Road,39460,53.777778,-1.572222,England
9,Leicester City,38,14,10,14,62,59,+3,52,50.8,32493,Leicester City,Leicester,Leicester,King Power Stadium,32500,52.620278,-1.142222,England


In [9]:
del merged_dataframe['Team']
del merged_dataframe['FDCOUK']
del merged_dataframe['Goal Difference']
del merged_dataframe['Country']
merged_dataframe

,Club Name,Matches Played,Wins,Draws,Loses,Goals For,Goals Against,Points,Expected Goals,Attendance,City,Stadium,Capacity,Latitude,Longitude
0,Arsenal,38,22,3,13,61,48,69,59.9,59665,London,Emirates Stadium,60361,51.555,-0.108611
1,Aston Villa,38,13,6,19,52,54,45,44.0,41681,Birmingham,Villa Park,42785,52.509167,-1.884722
2,Brentford,38,13,7,18,48,56,46,47.8,16918,London,Brentford Community Stadium,17250,51.489809,-0.287871
3,Brighton & Hove Albion,38,12,15,11,42,44,51,46.0,30966,Brighton,American Express Community Stadium,22374,50.861822,-0.083278
4,Burnley,38,7,14,17,34,53,35,39.0,19278,Burnley,Turf Moor,22546,53.789167,-2.230278
5,Chelsea,38,21,11,6,76,33,74,67.2,36443,London,Stamford Bridge,42449,51.481667,-0.191111
6,Crystal Palace,38,11,15,12,50,46,48,46.4,24588,London,Selhurst Park,26309,51.398333,-0.085556
7,Everton,38,11,6,21,43,66,39,42.8,38446,Liverpool,Goodison Park,40157,53.438889,-2.966389
8,Leeds United,38,9,11,18,42,79,38,47.2,36308,Leeds,Elland Road,39460,53.777778,-1.572222
9,Leicester City,38,14,10,14,62,59,52,50.8,32493,Leicester,King Power Stadium,32500,52.620278,-1.142222


In [10]:
token = "0574576bbef34e6ca8a079b111c70b41"
endpoint = "https://api.football-data.org/v4/competitions/PL/teams?season=2021"
headers = {'X-Auth-Token': token}
result = requests.get(endpoint, headers=headers).json()["teams"]
foundeds = dict()
for team in result:
    foundeds[team["name"][:-3]] = team["founded"]
sorted_foundeds = {k: foundeds[k] for k in sorted(foundeds)}
sorted_foundeds

{'Arsenal': 1886,
 'Aston Villa': 1872,
 'Brentford': 1889,
 'Brighton & Hove Albion': 1898,
 'Burnley': 1881,
 'Chelsea': 1905,
 'Crystal Palace': 1905,
 'Everton': 1878,
 'Leeds United': 1904,
 'Leicester City': 1884,
 'Liverpool': 1892,
 'Manchester City': 1880,
 'Manchester United': 1878,
 'Newcastle United': 1881,
 'Norwich City': 1902,
 'Southampton': 1885,
 'Tottenham Hotspur': 1882,
 'Watford': 1881,
 'West Ham United': 1895,
 'Wolverhampton Wanderers': 1877}

In [11]:
new_dataframe = pd.DataFrame(list(sorted_foundeds.items()),
                   columns=['Team', 'Founded'])
new_dataframe

,Team,Founded
0,Arsenal,1886
1,Aston Villa,1872
2,Brentford,1889
3,Brighton & Hove Albion,1898
4,Burnley,1881
5,Chelsea,1905
6,Crystal Palace,1905
7,Everton,1878
8,Leeds United,1904
9,Leicester City,1884


In [12]:
new_merged_dataframe = merged_dataframe.merge(new_dataframe, left_on='Club Name', right_on='Team')
new_merged_dataframe

,Club Name,Matches Played,Wins,Draws,Loses,Goals For,Goals Against,Points,Expected Goals,Attendance,City,Stadium,Capacity,Latitude,Longitude,Team,Founded
0,Arsenal,38,22,3,13,61,48,69,59.9,59665,London,Emirates Stadium,60361,51.555,-0.108611,Arsenal,1886
1,Aston Villa,38,13,6,19,52,54,45,44.0,41681,Birmingham,Villa Park,42785,52.509167,-1.884722,Aston Villa,1872
2,Brentford,38,13,7,18,48,56,46,47.8,16918,London,Brentford Community Stadium,17250,51.489809,-0.287871,Brentford,1889
3,Brighton & Hove Albion,38,12,15,11,42,44,51,46.0,30966,Brighton,American Express Community Stadium,22374,50.861822,-0.083278,Brighton & Hove Albion,1898
4,Burnley,38,7,14,17,34,53,35,39.0,19278,Burnley,Turf Moor,22546,53.789167,-2.230278,Burnley,1881
5,Chelsea,38,21,11,6,76,33,74,67.2,36443,London,Stamford Bridge,42449,51.481667,-0.191111,Chelsea,1905
6,Crystal Palace,38,11,15,12,50,46,48,46.4,24588,London,Selhurst Park,26309,51.398333,-0.085556,Crystal Palace,1905
7,Everton,38,11,6,21,43,66,39,42.8,38446,Liverpool,Goodison Park,40157,53.438889,-2.966389,Everton,1878
8,Leeds United,38,9,11,18,42,79,38,47.2,36308,Leeds,Elland Road,39460,53.777778,-1.572222,Leeds United,1904
9,Leicester City,38,14,10,14,62,59,52,50.8,32493,Leicester,King Power Stadium,32500,52.620278,-1.142222,Leicester City,1884


In [13]:
del new_merged_dataframe['Team']
new_merged_dataframe

,Club Name,Matches Played,Wins,Draws,Loses,Goals For,Goals Against,Points,Expected Goals,Attendance,City,Stadium,Capacity,Latitude,Longitude,Founded
0,Arsenal,38,22,3,13,61,48,69,59.9,59665,London,Emirates Stadium,60361,51.555,-0.108611,1886
1,Aston Villa,38,13,6,19,52,54,45,44.0,41681,Birmingham,Villa Park,42785,52.509167,-1.884722,1872
2,Brentford,38,13,7,18,48,56,46,47.8,16918,London,Brentford Community Stadium,17250,51.489809,-0.287871,1889
3,Brighton & Hove Albion,38,12,15,11,42,44,51,46.0,30966,Brighton,American Express Community Stadium,22374,50.861822,-0.083278,1898
4,Burnley,38,7,14,17,34,53,35,39.0,19278,Burnley,Turf Moor,22546,53.789167,-2.230278,1881
5,Chelsea,38,21,11,6,76,33,74,67.2,36443,London,Stamford Bridge,42449,51.481667,-0.191111,1905
6,Crystal Palace,38,11,15,12,50,46,48,46.4,24588,London,Selhurst Park,26309,51.398333,-0.085556,1905
7,Everton,38,11,6,21,43,66,39,42.8,38446,Liverpool,Goodison Park,40157,53.438889,-2.966389,1878
8,Leeds United,38,9,11,18,42,79,38,47.2,36308,Leeds,Elland Road,39460,53.777778,-1.572222,1904
9,Leicester City,38,14,10,14,62,59,52,50.8,32493,Leicester,King Power Stadium,32500,52.620278,-1.142222,1884


In [14]:
new_merged_dataframe.to_csv('./full_data_new.csv', index=False)

In [15]:
footbal_dataframe = pd.read_csv("./full_data_new.csv")
footbal_dataframe

,Club Name,Matches Played,Wins,Draws,Loses,Goals For,Goals Against,Points,Expected Goals,Attendance,City,Stadium,Capacity,Latitude,Longitude,Founded
0,Arsenal,38,22,3,13,61,48,69,59.9,59665,London,Emirates Stadium,60361,51.555000,-0.108611,1886
1,Aston Villa,38,13,6,19,52,54,45,44.0,41681,Birmingham,Villa Park,42785,52.509167,-1.884722,1872
2,Brentford,38,13,7,18,48,56,46,47.8,16918,London,Brentford Community Stadium,17250,51.489809,-0.287871,1889
3,Brighton & Hove Albion,38,12,15,11,42,44,51,46.0,30966,Brighton,American Express Community Stadium,22374,50.861822,-0.083278,1898
4,Burnley,38,7,14,17,34,53,35,39.0,19278,Burnley,Turf Moor,22546,53.789167,-2.230278,1881
5,Chelsea,38,21,11,6,76,33,74,67.2,36443,London,Stamford Bridge,42449,51.481667,-0.191111,1905
6,Crystal Palace,38,11,15,12,50,46,48,46.4,24588,London,Selhurst Park,26309,51.398333,-0.085556,1905
7,Everton,38,11,6,21,43,66,39,42.8,38446,Liverpool,Goodison Park,40157,53.438889,-2.966389,1878
8,Leeds United,38,9,11,18,42,79,38,47.2,36308,Leeds,Elland Road,39460,53.777778,-1.572222,1904
9,Leicester City,38,14,10,14,62,59,52,50.8,32493,Leicester,King Power Stadium,32500,52.620278,-1.142222,1884


In [16]:
engine = create_engine('sqlite:///english_football.db')

footbal_dataframe.to_sql(con=engine, index_label='id', name="team", if_exists='replace')

In [17]:
dataframe_from_sql = pd.read_sql("SELECT [Club Name], wins FROM team", engine)
dataframe_from_sql

,Club Name,Wins
0,Arsenal,22
1,Aston Villa,13
2,Brentford,13
3,Brighton & Hove Albion,12
4,Burnley,7
5,Chelsea,21
6,Crystal Palace,11
7,Everton,11
8,Leeds United,9
9,Leicester City,14
